# Tabelas

In [ ]:
pip install tabula-py

In [ ]:
import tabula
import pandas as pd
from flask import jsonify
import uuid
def generate_game_id():
    return str(uuid.uuid4())

In [ ]:
import csv
import json

def create_json(data, json_file):
    with open(json_file, 'w', encoding='utf-8') as file:  # Certifique-se de especificar a codificação correta
        json.dump(data, file, indent=4, ensure_ascii=False)  # Especifica ensure_ascii=False para lidar com acentos
    
    print('criado JSON em ' + json_file)

def csv_to_json(csv_file, json_file):
    """
    Converte um arquivo CSV em um arquivo JSON.

    :param csv_file: Caminho para o arquivo CSV.
    :param json_file: Caminho para o arquivo JSON de saída.
    """
    # Lista para armazenar os dados do CSV
    data = []

    # Lendo o arquivo CSV e populando a lista de dados
    with open(csv_file, 'r', encoding='utf-8') as file:  # Certifique-se de especificar a codificação correta
        csv_reader = csv.DictReader(file)
        for row in csv_reader:
            data.append(row)

    # Escrevendo os dados em um arquivo JSON
    create_json(data, json_file)

In [ ]:
def create_games_files(df_games, filepath):
    csv_file = filepath +'/games.csv'
    json_file = filepath +'/games.json'
    df_games.to_csv(csv_file, index=False)
    csv_to_json(csv_file, json_file)

# Criando tabela dos grupos

In [ ]:
def create_zero_ranking_group(table_groups, filepath):
    teams_groupA = table_groups['Grupo A'].tolist()
    teams_groupB = table_groups['Grupo B'].tolist()
    # Criar DataFrames vazios com as colunas necessárias
    columns = ['Time', 'Pontos', 'Jogos', 'V', 'E', 'D', 'Gols_Pro', 'Gols_Contra', 'Saldo']
    df_groupA = pd.DataFrame(columns=columns)
    df_groupB = pd.DataFrame(columns=columns)

    # Adicionar nomes dos grupos
    df_groupA['Time'] = teams_groupA
    df_groupB['Time'] = teams_groupB

    # Inicializar outras colunas com zeros
    for col in columns[1:]:
        df_groupA[col] = 0
        df_groupB[col] = 0

    # Define o tipo de dados para as colunas
    dtypes = {'Pontos': int, 'Jogos': int, 'V': int, 'E': int, 'D': int, 'Gols_Pro': int, 'Gols_Contra': int, 'Saldo': int}
    df_groupA = df_groupA.astype(dtypes)
    df_groupB = df_groupB.astype(dtypes)
    df_groupA['ID'] = [generate_game_id() for _ in range(len(df_groupA))]
    df_groupB['ID'] = [generate_game_id() for _ in range(len(df_groupB))]
    
    df_groupA.to_csv(filepath + '/ranking_A.csv', index=False)
    df_groupB.to_csv(filepath  + '/ranking_B.csv', index=False)
    
    csv_to_json(filepath  + '/ranking_A.csv', filepath + '/ranking_A.json')
    csv_to_json(filepath + '/ranking_B.csv', filepath + '/ranking_B.json')

    return df_groupA, df_groupB

# Formatando as tabelas dos grupos

In [ ]:
def format_table_group(tb_groupA, tb_groupB):
    df_groupA_base = pd.DataFrame(tb_groupA)
    df_groupB_base = pd.DataFrame(tb_groupB)

    # Atualizando nome das colunas:
    novos_nomes_colunas = ['Col.', 'Atléticas', 'Pontos', 'Jogos', 'V', 'E', 'D', 'Gols Pró', 'Gols Contra', 'Saldo']
    df_groupA_base.columns = novos_nomes_colunas
    df_groupB_base.columns = novos_nomes_colunas


    # Removendo a primeira coluna
    df_groupA = df_groupA_base.drop(columns=['Col.'])
    df_groupB = df_groupB_base.drop(columns=['Col.'])

    # Removendo as duas primeiras linhas (antiga coluna)
    df_groupA.drop([0, 1], inplace=True)
    df_groupB.drop([0, 1], inplace=True)


    # Converter os dados das colunas para o tipo int
    colunas_para_converter = ['Pontos', 'Jogos', 'V', 'E', 'D', 'Gols Pró', 'Gols Contra', 'Saldo']
    df_groupA[colunas_para_converter] = df_groupA[colunas_para_converter].astype(int)
    df_groupB[colunas_para_converter] = df_groupB[colunas_para_converter].astype(int)


    print("Grupo A:")
    print(df_groupA)
    print("#########################################################################")
    print("Grupo B:")
    print(df_groupB)

# Formatando tabela de jogos

In [ ]:
def generate_table_games(tb_games):
    df_games = pd.DataFrame(tb_games)
    if df_games['DIA'].isna().all():
        df_games['DIA'] = ''
    else:
        # Limpar os espaços extras e formatar a coluna 'DIA'
        df_games['DIA'] = df_games['DIA'].str.replace(' ', '', regex=False)
        df_games['DIA'] = pd.to_datetime(df_games['DIA'] + '/' + pd.to_datetime('now', utc=True).strftime('%Y'), format='%d/%m/%Y', errors='coerce')

    # Limpar os espaços extras e corrigir a formatação dos placares
    df_games['PLACAR'] = df_games['PLACAR'].str.replace(' ', '', regex=True)
    df_games[['GOLS_MANDANTE', 'GOLS_VISITANTE']] = df_games['PLACAR'].str.split('X', expand=True)
    df_games['SIMULADOR'] = False
    df_games.rename(columns={'HORÁRIO': 'HORARIO'}, inplace=True)
    df_games.rename(columns={'EQUIPE Mandante': 'Mandante'}, inplace=True)
    df_games.rename(columns={'EQUIPE Visitante': 'Visitante'}, inplace=True)
    
    print(df_games)
    return df_games

# Jogos por time

In [ ]:
def listar_jogos_por_time(df_games, team_surname):
  condition_home = df_games['EQUIPE Mandante'].str.contains(team_surname)
  condition_away = df_games['EQUIPE Visitante'].str.contains(team_surname)
  games_by_team = df_games[condition_home | condition_away]
  return games_by_team

def listar_confrontos(df_games, teamOne, teamTwo):
    # Filtrar os jogos onde a equipe é a mandante ou visitante
    condition_home = df_games['EQUIPE Mandante'].str.contains(teamOne)
    condition_away = df_games['EQUIPE Visitante'].str.contains(teamOne)

    # Combinar as condições usando o operador lógico OR (|)
    games_teamOne = df_games[condition_home | condition_away]

    # Filtrar os jogos onde a outra equipe é a visitante
    games_between_home = games_teamOne[games_teamOne['EQUIPE Mandante'].str.contains(teamTwo)]
    if games_between_home.empty == False:
      return games_between_home
    else:
      return games_teamOne[games_teamOne['EQUIPE Visitante'].str.contains(teamTwo)]

# Confronto direto

In [ ]:
def update_confronto_direto(winner_team, loser_team, draw, confrontos_diretos):
   if draw:
      resultado = 'E'
   else:
      resultado = winner_team

  # Registrar o resultado no dicionário
   confrontos_diretos.setdefault(winner_team, {}).setdefault(loser_team, resultado)
   confrontos_diretos.setdefault(loser_team, {}).setdefault(winner_team, resultado)
   return confrontos_diretos

def remover_confronto_direto(winner_team, loser_team, confrontos_diretos):
  # Registrar o resultado no dicionário
   confrontos_diretos.setdefault(winner_team, {}).setdefault(loser_team, '')
   confrontos_diretos.setdefault(loser_team, {}).setdefault(winner_team, '')
   return confrontos_diretos

def gerar_confronto_direto(df_games, filepath):
  confrontos_diretos = {}

  # Iterar sobre os jogos e registrar os resultados dos confrontos diretos
  for _, jogo in df_games.iterrows():
      equipe_mandante = jogo['Mandante']
      equipe_visitante = jogo['Visitante']
      placar_mandante = jogo['GOLS_MANDANTE']
      placar_visitante = jogo['GOLS_VISITANTE']

      resultado = ''

      if placar_mandante > placar_visitante:
          resultado = equipe_mandante
      elif placar_mandante < placar_visitante:
          resultado = equipe_visitante
      else:
          resultado = 'E'

      # Registrar o resultado no dicionário
      confrontos_diretos.setdefault(equipe_mandante, {}).setdefault(equipe_visitante, resultado)
      confrontos_diretos.setdefault(equipe_visitante, {}).setdefault(equipe_mandante, resultado)

  create_json(confrontos_diretos, filepath + '/confrontation.json')

  return confrontos_diretos

def confrontos_to_df(confrontos_diretos):
  # Criar um DataFrame a partir dos resultados dos confrontos diretos
  df_confrontos_diretos = pd.DataFrame(confrontos_diretos).T.fillna('').sort_index()
  df_confrontos_diretos.index.name = 'Equipes'
  return df_confrontos_diretos

# Filtro com classificados

In [ ]:
def atualizar_classificacao(group, df_confrontos_diretos):
  if group == 'A':
    df_group = df_groupA
  elif group == 'B':
    df_group = df_groupB

  linhas_pontos_iguais = df_group[df_group.duplicated(subset='Pontos', keep=False)]

  # Obter apenas os nomes das atléticas
  atléticas_pontos_iguais = linhas_pontos_iguais['Atléticas'].tolist()
  if len(atléticas_pontos_iguais) == 2:
    team_ahead = df_confrontos_diretos.loc[atléticas_pontos_iguais[0], atléticas_pontos_iguais[1]]
    atléticas_pontos_iguais.remove(team_ahead)

    # Nome da atlética que você deseja trocar
    team_behind = atléticas_pontos_iguais[0]

    # Encontrar a posição da atlética no DataFrame
    position_ahead = df_group.index[df_group['Atléticas'] == team_ahead].tolist()[0]
    position_behind = df_group.index[df_group['Atléticas'] == team_behind].tolist()[0]

    # Trocar os valores entre as linhas diretamente
    df_group.loc[position_ahead], df_group.loc[position_behind] = df_group.loc[position_behind].copy(), df_group.loc[position_ahead].copy()

  print(f"Classificação grupo {group} atualizada com confrontos diretos:")
  print(df_group.sort_values(by='Pontos', ascending=False))

# Simulação

In [ ]:
def atualizar_df_games(new_game_data):
    global df_games
    df_games = pd.concat([df_games, pd.DataFrame([new_game_data])], ignore_index=True)

def remover_df_games(matches_to_remove, simulador=True):
    # Obter os índices dos jogos a serem removidos
    indices_to_remove = matches_to_remove.index

    # Remover os jogos do DataFrame original
    df_games.drop(indices_to_remove, inplace=True)

# Simular o resultado do jogo (exemplo simples)
def simular_jogo(group, home_team, home_goal, away_team, away_goal, confrontos_diretos):
    
    if listar_confrontos(home_team, away_team).empty == False:
        print("Não posso substituir um jogo que já existe")
    else:
        game = {
            'GRUPO': group,
            'EQUIPE Mandante': home_team,
            'GOLS_MANDANTE': home_goal,
            'EQUIPE Visitante': away_team,
            'GOLS_VISITANTE': away_goal,
            'PLACAR': str(home_goal) + 'x' + str(away_goal),
            'SIMULADOR': 1
        }

        # Definir df por grupo
        if group == 'A':
          df_group = df_groupA
        elif group == 'B':
          df_group = df_groupB

        condition_home = df_group['Atléticas'] == home_team
        condition_away = df_group['Atléticas'] == away_team

    # Atualizar o número de jogos
        df_group.loc[condition_home, 'Jogos'] += 1
        df_group.loc[condition_away, 'Jogos'] += 1

    # Atualizar os gols
        df_group.loc[condition_home, 'Gols Pró'] += home_goal
        df_group.loc[condition_home, 'Gols Contra'] += away_goal

        df_group.loc[condition_away, 'Gols Pró'] += away_goal
        df_group.loc[condition_away, 'Gols Contra'] += home_goal

        df_group.loc[condition_home, 'Saldo'] += home_goal - away_goal
        df_group.loc[condition_away, 'Saldo'] += away_goal - home_goal

    # Atualizar os pontos
        if home_goal == away_goal:
            df_group.loc[condition_home, 'Pontos'] += 1
            df_group.loc[condition_away, 'Pontos'] += 1
            df_group.loc[condition_home, 'E'] += 1
            df_group.loc[condition_away, 'E'] += 1
            update_confronto_direto(away_team, home_team, True, confrontos_diretos)
        elif home_goal > away_goal:
            df_group.loc[condition_home, 'Pontos'] += 3
            df_group.loc[condition_home, 'V'] += 1
            df_group.loc[condition_away, 'D'] += 1
            update_confronto_direto(home_team, away_team, False, confrontos_diretos)
        else:
            df_group.loc[condition_away, 'Pontos'] += 3
            df_group.loc[condition_away, 'V'] += 1
            df_group.loc[condition_home, 'D'] += 1
            update_confronto_direto(away_team, home_team, False, confrontos_diretos)

        atualizar_df_games(game)
        atualizar_classificacao(group, confrontos_to_df(confrontos_diretos))

In [ ]:
def remover_jogo(home_team, away_team, confrontos_diretos):

    df_confronto = listar_confrontos(home_team, away_team)
#   Se for jogo simulado  
    if df_confronto['SIMULADOR'].tolist()[0]:
        group = df_confronto['GRUPO'].values[0]

        # Definir df por grupo
        if group == 'A':
            df_group = df_groupA
        elif group == 'B':
            df_group = df_groupB

        home_team = df_confronto['EQUIPE Mandante'].tolist()[0]
        away_team = df_confronto['EQUIPE Visitante'].tolist()[0]
        home_goal = int(df_confronto['GOLS_MANDANTE'].tolist()[0])
        away_goal = int(df_confronto['GOLS_VISITANTE'].tolist()[0])

        condition_home = df_group['Atléticas'] == home_team
        condition_away = df_group['Atléticas'] == away_team

    # Atualizar o número de jogos
        df_group.loc[condition_home, 'Jogos'] -= 1
        df_group.loc[condition_away, 'Jogos'] -= 1

    # Atualizar os gols
        df_group.loc[condition_home, 'Gols Pró'] -= home_goal
        df_group.loc[condition_home, 'Gols Contra'] -= away_goal

        df_group.loc[condition_away, 'Gols Pró'] -= away_goal
        df_group.loc[condition_away, 'Gols Contra'] -= home_goal

        df_group.loc[condition_home, 'Saldo'] += home_goal - away_goal
        df_group.loc[condition_away, 'Saldo'] += away_goal - home_goal

    # Atualizar os pontos
        if home_goal == away_goal:
            df_group.loc[condition_home, 'Pontos'] -= 1
            df_group.loc[condition_away, 'Pontos'] -= 1
            df_group.loc[condition_home, 'E'] -= 1
            df_group.loc[condition_away, 'E'] -= 1
        elif home_goal > away_goal:
            df_group.loc[condition_home, 'Pontos'] -= 3
            df_group.loc[condition_home, 'V'] += 1
            df_group.loc[condition_away, 'D'] += 1
        else:
            df_group.loc[condition_away, 'Pontos'] -= 3
            df_group.loc[condition_away, 'V'] += 1
            df_group.loc[condition_home, 'D'] += 1

        remover_df_games(df_confronto)
        remover_confronto_direto(home_team, away_team, confrontos_diretos)
        atualizar_classificacao(group, confrontos_to_df(confrontos_diretos))
    else:
        print('Nenhum jogo simulado foi encontrado')

In [ ]:
import uuid

# Função para gerar IDs
def generate_game_id():
    return str(uuid.uuid4())

df_games['ID'] = [generate_game_id() for _ in range(len(df_games))]
df_games.rename(columns={'HORÁRIO': 'HORARIO'}, inplace=True)
df_games.rename(columns={'EQUIPE Mandante': 'Mandante'}, inplace=True)
df_games.rename(columns={'EQUIPE Visitante': 'Visitante'}, inplace=True)

In [ ]:
df_games.to_csv('files/games.csv', index=False)

In [ ]:
csv_file = 'files/FM.csv'
json_file = 'files/games.json'
csv_to_json(csv_file, json_file)

# Execução

In [ ]:
dic_modalities_page = {
    "FF/A" : "42-43",
    "FF/B" : "46-47",
    "FF/C" : "50-51",
    "FF/D" : "54-55",
    "FF/E" : "58-59",
    "FM/A" : "62-63",
    "FM/B" : "66-67",
    "FM/C" : "70-71",
    "FM/D" : "74-75",
    "FM/E" : "78-79",
    "FM/F" : "82-83",
}

In [ ]:
modality = 'FM/D'
tables = tabula.read_pdf("files/Boletim.pdf", pages=dic_modalities_page[modality])
print(tables)
tb_group = tables[0]
tb_games = tables[1]

## Zeradas

In [ ]:
filepath = 'files/' + modality
filepath_group = filepath + '/group'
rankings = create_zero_ranking_group(tb_group, filepath_group)
df_games = generate_table_games(tb_games)

### Dicionário de correção

In [ ]:
locations = ['Palestra', 'Idalina', 'Pinheiros', 'SEMEF', 'GETA', 'EDA', 'CESPRO']
correction_local = {
    'Pale stra': 'Palestra',
    'Idal ina': 'Idalina',
    'SEM EF': 'SEMEF',
}
# Função de validação e correção
def correct_local(local):
    if local in locations:
        return local
    elif local in correction_local:
        return correction_local[local]
    else:
        return local

# Aplicar a função de validação e correção nas colunas 'EQUIPE Mandante' e 'EQUIPE Visitante'
df_games['LOCAL'] = df_games['LOCAL'].apply(correct_local)
print(df_games['LOCAL'].unique())

In [ ]:
teams_groupA = tb_group['Grupo A'].tolist()
teams_groupB = tb_group['Grupo B'].tolist()
teams = teams_groupA + teams_groupB
correction_teams = {
    'ArquiteturaMackenzie': 'Arquitetura Mackenzie',
    'Belas A rtes': 'Belas Artes',
    'CAAP U FABC': 'CAAP UFABC',
    'Comunica ção PUC': 'Comunicação PUC',
    'Comunicaçã o Anhembi': 'Comunicação Anhembi',
    'Comunicaçã o Mackenzie': 'Comunicação Mackenzie',
    'Comunicaçã o Metodista': 'Comunicação Metodista',
    'Direit o PUC': 'Direito PUC',
    'Direit o USP': 'Direito USP',
    'Direito M ackenzie': 'Direito Mackenzie',
    'Direito S ã o Judas': 'Direito São Judas',
    'Direito S ão Judas': 'Direito São Judas',
    'Direito SãoBernardo': 'Direito São Bernardo',
    'EACHUSP': 'EACH USP',
    'Economia M ackenzie': 'Economia Mackenzie',
    'Economia M ackenzie': 'Economia Mackenzie',
    'Educação Fís i ca Anhembi': 'Educação Física Anhembi',
    'Educação Fís i ca UNINOVE': 'Educação Física UNINOVE',
    'Educação Fís ica Anhembi': 'Educação Física Anhembi',
    'EngenhariaAnhembi': 'Engenharia Anhembi',
    'EngenhariaMackenzie': 'Engenharia Mackenzie',
    'EngenhariaSão Judas': 'Engenharia São Judas',
    'ESP M': 'ESPM',
    'Farmác ia USP': 'Farmácia USP',
    'Fatec Sã o Paulo': 'Fatec São Paulo',
    'FAUUSP': 'FAU USP',
    'FE I': 'FEI',
    'FEA P UC': 'FEA PUC',
    'FEA U SP': 'FEA USP',
    'FEC AP': 'FECAP',
    'Federaldo ABC': 'Federal do ABC',
    'FM U': 'FMU',
    'GetúlioVargas': 'Getúlio Vargas',
    'IBME C SP': 'IBMEC SP',
    'INS P ER': 'INSPER',
    'INSP ER': 'INSPER',
    'IT A': 'ITA',
    'LEP Mac kenzie': 'LEP Mackenzie',
    'LEP MAC KENZIE': 'LEP MACKENZIE',
    'Medici n a USP': 'Medicina USP',
    'Medicin a Santos': 'Medicina Santos',
    'Medicina PU C Campinas': 'Medicina PUC Campinas',
    'Medicina S ã o Caetano': 'Medicina São Caetano',
    'Medicina S anta Casa': 'Medicina Santa Casa',
    'Medicina Sa nto Amaro': 'Medicina Santo Amaro',
    'Medicina Sã o Bernardo': 'Medicina São Bernardo',
    'Medicina Sã o Caetano': 'Medicina São Caetano',
    'Medicina San  ta Marcelina': 'Medicina Santa Marcelina',
    'Medicina San ta Marcelina': 'Medicina Santa Marcelina',
    'Medicina Ta ubaté (DT)': 'Medicina Taubaté',
    'Medicina U NICAMP': 'Medicina UNICAMP',
    'MedicinaAnhembi': 'Medicina Anhembi',
    'MedicinaBragança': 'Medicina Bragança',
    'MedicinaEinstein': 'Medicina Einstein',
    'MedicinaJundiaí': 'Medicina Jundiaí',
    'MedicinaTaubaté': 'Medicina Taubaté',
    'MedicinaUNICID': 'Medicina UNICID',
    'MedicinaUNIMES': 'Medicina UNIMES',
    'MedicinaZ Taubaté': 'Medicina Taubaté',
    'Politécn ica USP': 'Politécnica USP',
    'Psicologia  d a PUC SP': 'Psicologia da PUC SP',
    'Psicologia d a PUC SP': 'Psicologia da PUC SP',
    'SEN  AC': 'SENAC',
    'SEN AC': 'SENAC',
    'Sistema de Inf ormação USP': 'Sistema de Informação USP',
    'UN IP': 'UNIP',
    'UNIFESP D iadema': 'UNIFESP Diadema',
    'Unifesp S ão Paulo': 'Unifesp São Paulo',
    'UNIFESPOsasco': 'UNIFESP Osasco',
    'US C S': 'USCS',
    'US CS': 'USCS',
    'Veteriná ria USP': 'Veterinária USP',
}
# Função de validação e correção
def correct_team(equipe):
    equipe_strip = equipe.strip().replace('  ', ' ')
    if equipe_strip in teams:
        return equipe_strip
    elif equipe_strip in correction_teams:
        return correction_teams[equipe_strip]
    else:
        return equipe_strip

# Aplicar a função de validação e correção nas colunas 'EQUIPE Mandante' e 'EQUIPE Visitante'
df_games['Mandante'] = df_games['Mandante'].apply(correct_team)
df_games['Visitante'] = df_games['Visitante'].apply(correct_team)
print(df_games['Mandante'].unique())
print(df_games['Visitante'].unique())

In [ ]:
create_games_files(df_games, filepath)
gerar_confronto_direto(df_games, filepath)

## Atualização